In [1]:
# Input info
# se_f = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/SE.rds"
# minCell = 1
# outdir = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/clone_counts/minCellConds_1" 
# sample_names = "pre,post"

se_f = "/data/Mito_Trace/output/pipeline/v02/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/knn/kparam_30/gff_A2_black/annotation_clones/SE.rds"
outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/knn/kparam_30/gff_A2_black/annotation_clones/clone_counts/minCellConds_1"
sample_names = "Input" 
minCell = 1 


# Parallel processing
cores = 4

# R utils script
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"



In [2]:
source(utils_script)
#source("/data/Mito_Trace/R_scripts/plot_de_over_all.R")

Attaching SeuratObject


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
dir.create(outdir, showWarnings = TRUE)

Warning message in dir.create(outdir, showWarnings = TRUE):
“cannot create dir '/data/Mito_Trace/output/pipeline/v02/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/knn/kparam_30/gff_A2_black/annotation_clones/clone_counts/minCellConds_1', reason 'No such file or directory'”


In [4]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
sample_names

[1] "Input"

In [5]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")
library(dplyr)
library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:dplyr’:

    first, rename


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package:

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [6]:
library(dplyr)

In [7]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
se

An object of class Seurat 
112098 features across 4657 samples within 2 assays 
Active assay: RNA (19039 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: lsi, umap

# Get clones with mininum  cells in each condition and run DE pre and post

In [8]:
source(utils_script)
tmp = get.clone.sizes(se)
counts.norm=tmp[["counts.norm"]]
clone.sizes = tmp[["clone.size"]]
head(clone.sizes)

`summarise()` has grouped output by 'condition', 'donor', 'lineage'. You can override using the `.groups` argument.
Warning message in xtfrm.data.frame(x):
"cannot xtfrm data frames"
`summarise()` has grouped output by 'donor'. You can override using the `.groups` argument.


donor,lineage,norm.total,total,name,cdf.norm,cdf,index
<fct>,<fct>,<dbl>,<int>,<fct>,<dbl>,<int>,<int>
0,33,0.06652452,156,0_33,0.03326226,156,1
0,0,0.05117271,120,0_0,0.05884861,276,2
0,1,0.05074627,119,0_1,0.08422175,395,3
0,2,0.04264392,100,0_2,0.10554371,495,4
0,3,0.03923241,92,0_3,0.12515991,587,5
0,4,0.03795309,89,0_4,0.14413646,676,6


In [9]:
curr.counts.norm = counts.norm[counts.norm$size>minCell,]
tmp.thresh = curr.counts.norm %>% group_by(name) %>% summarize(size=sum(size))
curr.counts.norm = curr.counts.norm[tmp.thresh$size > 1, ]
curr.counts.norm = curr.counts.norm[order(curr.counts.norm$name),]
topClones = unique(curr.counts.norm$name)
#curr.counts.norm

## Save

In [10]:
out.curr.counts.norm = tidyr::pivot_wider(curr.counts.norm, id_cols="name",values_from="norm", names_from="condition")
out.curr.counts.norm$size.mean = rowMeans(out.curr.counts.norm %>% dplyr::select(dplyr::all_of(sample_names)))

if(length(sample_names)==1){
    out.curr.counts.norm$fold = 1
}else{
    out.curr.counts.norm$fold =  out.curr.counts.norm[[sample_names[1]]]/out.curr.counts.norm[[sample_names[2]]]
    #out.curr.counts.norm = out.curr.counts.norm %>% arrange(desc(size.mean)) %>% dplyr::select(-c(size.mean))
    }
out.curr.counts.norm = out.curr.counts.norm %>% arrange(desc(fold))  %>% dplyr::select(-c(size.mean))
ord.inds = as.character(out.curr.counts.norm[["name"]])
out.curr.counts.norm



name,Input,fold
<fct>,<dbl>,<dbl>
0_0,0.05117271,1
0_1,0.05074627,1
0_10,0.03198294,1
0_11,0.03027719,1
0_12,0.03070362,1
0_13,0.02985075,1
0_14,0.02985075,1
0_15,0.02601279,1
0_16,0.02558635,1


In [11]:
out.curr.counts = tidyr::pivot_wider(curr.counts.norm, id_cols="name",values_from="total", names_from="condition")
out.curr.counts = as.data.frame(out.curr.counts)
row.names(out.curr.counts) = out.curr.counts$name
out.curr.counts = out.curr.counts[ord.inds,]
out.curr.counts

,name,Input
,<fct>,<int>
0_0,0_0,2345
0_1,0_1,2345
0_10,0_10,2345
0_11,0_11,2345
0_12,0_12,2345
0_13,0_13,2345
0_14,0_14,2345
0_15,0_15,2345
0_16,0_16,2345


In [12]:
readr::write_csv(out.curr.counts.norm, file.path(outdir, "clone_sizes_norm.csv"))
readr::write_csv(out.curr.counts, file.path(outdir, "clone_sizes.csv"))

ERROR: Error: Cannot open file for writing:
* '/data/Mito_Trace/output/pipeline/v02/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/knn/kparam_30/gff_A2_black/annotation_clones/clone_counts/minCellConds_1/clone_sizes_norm.csv'


In [ ]:
ggplot(data=curr.counts.norm, aes(x=name,y=norm, fill = condition)) + 
        geom_col(position='dodge') 
    

ggsave(file.path(outdir, "topClonesSize_NormBatch.pdf"))

In [ ]:
ggplot(data=curr.counts.norm, aes(x=name,y=size, fill = condition)) + 
        geom_col(position='dodge') 
    

ggsave(file.path(outdir, "topClonesSize.pdf"))

In [ ]:
sessionInfo()

In [ ]:
outdir